In [78]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 熵权法计算权重

In [79]:
# 读取数据
data = pd.read_excel('../Resource/ExperimentDataBase/Spad_Dta_GRX.xlsx')

In [80]:
data_n_data = data[['SPAD','AGB','LAI','Ci']].values

In [81]:
data.head()

,SPAD,AGB,LAI,Pn,Ci,Tr,Gs
0,37.466667,1.708889,2.917709,16.573453,263.358997,3.381668,0.178455
1,35.991667,2.063889,2.054150,22.759493,268.242120,5.887681,0.395477
2,35.941667,1.997500,3.599002,16.718943,220.002423,4.542420,0.274608
3,34.941667,1.362222,3.320240,18.022922,196.736997,3.629510,0.170752
4,40.300000,1.965000,3.309667,17.883205,215.082727,4.303128,0.226866


In [104]:
def entropy_weight(matrix):
    """
    计算熵权法权重
    :param matrix: 决策矩阵，shape为(m, n)，m为样本数，n为指标数
    :return: 各指标的权重，shape为(n,)
    """
    m, n = matrix.shape
    # 使用归一化处理
    norm_matrix = (matrix - matrix.min(axis=0)) / (matrix.max(axis=0) - matrix.min(axis=0))
    #print(norm_matrix)
    # 计算信息熵
    norm_matrix[norm_matrix == 0] = 1e-10 # 将零值替换为一个较小的非零值
    entropy = - np.sum(norm_matrix * np.log(norm_matrix), axis=0)
    print(entropy)
    # 计算权重
    weight = (1 - entropy) / np.sum(1 - entropy)
    return weight

### 第一种归一化方式使用最大值和最小值进行缩放，通过除以最大值和最小值之间的差，将数据缩放到0到1之间，即：
- Xnorm = X-Xmin / Xmax - Xmin
- 其中x 是原始数据，Xnorm 是归一化后的数据，Xmin 和 Xmax  分别是该列的最小值和最大值。
### 第二种归一化方式是将每一列数据除以该列数据的和，从而将数据缩放到0到1之间，即：
- Xnorm  = X / Σ(i=1->n)Xi
- 其中 x 是原始数据， Xnorm 是归一化后的数据，n 是该列数据的数量。
> 第一种归一化方式将每个特征的范围缩放到相同的区间，但是可能会受到极值的影响。第二种归一化方式则保证了每个特征的和为1，不受极值的影响，但是可能会受到各特征数量的影响。

In [105]:
# 计算决策矩阵X和归一化矩阵Y
# X = data_n_data.iloc[:, :].values
# Y = np.apply_along_axis(lambda x: x / np.sum(x), 0, X)

In [106]:
# 计算信息熵和权重
# m, n = Y.shape
# E = (-1 / np.log(m)) * np.sum(Y * np.log(Y), axis=0)
# W = (1 - E) / np.sum(1 - E)

In [107]:
# 归一化处理
# W = W / np.sum(W)

In [108]:
# 输出结果
# for i, w in enumerate(W):
#     print(f"第{i+1}列的权重为{w:.3f}")

In [109]:
# 计算指标权重
weights = entropy_weight(data_n_data)
print(weights)

[20.80267728 16.78550941 19.11857991 19.52615131]
[0.27415032 0.21853623 0.2508355  0.25647796]


In [110]:
# 输出各指标权重
print("指标权重：")
for i in range(len(weights)):
    print("{}: {:.3f}".format(data.columns[i], weights[i]))

指标权重：
SPAD: 0.274
AGB: 0.219
LAI: 0.251
Pn : 0.256
